## Image Inpainting using OpenVINO
This notebook demonstrates how to use gmcnn image inpainting model with OpenVINO.
The Following pipeline will be created in this notebook.
<img align='center' src="data/pipeline.png" alt="drawing" width="600"/>


This model is used to obtain something very similar to the original image given a tampered image.
More details about the [GMCNN model](https://github.com/shepnerd/inpainting_gmcnn)

In [ ]:
import os
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
from openvino.runtime import Core

### Downloading the Models
Models can be downloaded from omz downloader. omz is a command line tool for downloading models from the open model zoo.
`gmcnn-places2-tf` is the omz name for the considered model. You can find the names of available models [here](https://github.com/openvinotoolkit/open_model_zoo/blob/master/models/public/index.md) and [here](https://github.com/openvinotoolkit/open_model_zoo/blob/master/models/intel/index.md)


In [ ]:
# Directory where model will be downloaded
base_model_dir = "model"
# Model name as named in Open Model Zoo
model_name = "gmcnn-places2-tf"

model_path = Path(f"{base_model_dir}/public/{model_name}/{model_name}/frozen_model.pb")
if not model_path.exists():
    download_command = f"omz_downloader " \
                       f"--name {model_name} " \
                       f"--output_dir {base_model_dir}"
    ! $download_command
else:
    print("Already downloaded")

### Convert Tensorflow model to OpenVINO IR format

We will be using the model optimizer command line tool for converting the tensorflow model to IR format. 

We will extract the necessary information from [the overview of openvino models](https://docs.openvino.ai/latest/omz_models_model_gmcnn_places2_tf.html) (alternatively you can check the openvino/open_model_zoo as well)

`input_model` path to the tensorflow model.

`input shape` input shape of the model, in this case we have 2 inputs given with a commaa seperating the 2 shapes.

`input` Used to give the input names. This is essential here because we have 2 inputs.

`output_dir` Output directory/name.

For more details about the parameters please check [here](https://docs.openvino.ai/latest/openvino_docs_MO_DG_prepare_model_convert_model_Converting_Model.html)



In [ ]:
precision = "FP32"
ir_path = Path(f"{base_model_dir}/public/{model_name}/{precision}/{model_name}.xml")

# Run Model Optimizer if the IR model file does not exist
if not ir_path.exists():
    print("Exporting TensorFlow model to IR... This may take a few minutes.")
    convert_command = f"omz_converter " \
                      f"--name {model_name} " \
                      f"--download_dir {base_model_dir} " \
                      f"--precisions {precision}"
    ! $convert_command
else:
    print("IR model already exists.")

### Load the model

Now we will load the IR formatted model.

 1. Initialize inference engine (IECore)
 2. Read the network from *.bin and *.xml files (weights and architecture)
 3. Load the model on the "CPU."
 4. Get input and output names of nodes.

Only a few lines of code are required to run the model. Let's see it.

In [ ]:
core = Core()

# Read the model.xml and weights file
model = core.read_model(model=ir_path)
# load the model on to the CPU
compiled_model = core.compile_model(model=model, device_name="CPU")
# Store the input node names as a list because this model has 2 inputs
input_layers = list(compiled_model.inputs)
output_layer = compiled_model.output(0)

### Determine the input shapes of the model

Lets save input shapes into a list called `input_shapes` 
Note that both the image dimentions are same however the second input shape has a channel of 1 (monotone)

*since input dimentions are used for resizing we have copied it to H and W variables

In [ ]:
N, H, W, C = input_layers[0].shape

### Create a square mask

Next we will create a single channeled mask that will be laid on top of the original image

In [ ]:
def create_mask(image_width, image_height, size_x=30, size_y=30, number=1):
    """
    Create a square mask of defined size on a random location

    :param: image_width: width of the image
    :param: image_height: height of the image
    :param: size: size in pixels of one side
    :returns:
            mask: grayscale float32 mask of size [image_height,image_width,1]
    """
    
    mask = np.zeros((image_height, image_width, 1), dtype=np.float32)
    for _ in range(number):
        start_x = np.random.randint(image_width - size_x)
        start_y = np.random.randint(image_height - size_y)
        cv2.rectangle(img=mask, pt1=(start_x, start_y), pt2=(start_x + size_x, start_y + size_y), color=(1, 1, 1), thickness=cv2.FILLED)
    return mask

In [ ]:
# Generate a square mask of size WxH with number of "holes"
mask = create_mask(image_width=W, image_height=H, size_x=50, size_y=50, number=10)
# This mask will be laid over the input image as noise
plt.imshow(cv2.cvtColor(mask, cv2.COLOR_BGR2RGB));

### Load and Resize the Image

This image will be altered using a mask. 

In [ ]:
# Load Image
image = cv2.imread("data/test_image.png")
# Resize image to meet network expected input sizes
resized_image = cv2.resize(src=image, dsize=(W, H), interpolation=cv2.INTER_AREA)
plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB));

### Generating the Masked Image

This is the multiplication of the image and the mask gives us the result of the masked image layered on top of the original image. 

The `masked_image` will be the first input to GMCNN model

In [ ]:
# Generating masked image
masked_image = (resized_image * (1 - mask) + 255 * mask).astype(np.uint8)
plt.imshow(cv2.cvtColor(masked_image, cv2.COLOR_BGR2RGB));

### Preprocessing function to aling with the ```input_shapes```

The model expects the input dimentions to be NCHW.

- masked_image.shape = (512,680,3) -----> model excepts = (1,3,512,680)
- resized_mask.shape = (512,680,1) -----> model excepts = (1,1,512,680)

In [ ]:
masked_image = masked_image[None, ...]
mask = mask[None, ...]

In [ ]:
result = compiled_model([masked_image, mask])[output_layer]
plt.imshow(cv2.cvtColor(result.squeeze().astype(np.uint8), cv2.COLOR_BGR2RGB))